# 08 — Temporal Robustness Study (Kaggle GPU)

This notebook runs the temporal robustness experiments:
1. Install dependencies (PyG).
2. Copy the repo snapshot (`elliptic-gnn-baselines`) into `/kaggle/working`.
3. Copy the Elliptic dataset (`elliptic-fraud-detection`) into `data/Elliptic++ Dataset`.
4. Set deterministic seeds.
5. Regenerate `splits.json` (safety).
6. Run `scripts/run_m9_temporal_shift.py`.
7. Inspect `reports/m9_temporal_results.csv`.

Attach two datasets via Kaggle's "Add data":
- `elliptic-gnn-baselines` (repo snapshot)
- `elliptic-fraud-detection` (Elliptic++ data folder)


## 1. Install Dependencies

In [ ]:
!pip install torch-geometric -q

## 2. Imports & Workspace Setup

In [ ]:
import os
import shutil
import sys
from pathlib import Path

REPO_DATASET = Path('/kaggle/input/elliptic-gnn-baselines')
WORKDIR = Path('/kaggle/working/elliptic-gnn-baselines')
if REPO_DATASET.exists():
    if WORKDIR.exists():
        shutil.rmtree(WORKDIR)
    shutil.copytree(REPO_DATASET, WORKDIR)
    nested = WORKDIR / 'elliptic-gnn-baselines'
    if nested.exists():
        os.chdir(nested)
    else:
        os.chdir(WORKDIR)
else:
    print('WARNING: Add the ellliptic-gnn-baselines dataset.')

if str(Path.cwd()) not in sys.path:
    sys.path.insert(0, str(Path.cwd()))

DATASET_SRC = Path('/kaggle/input/elliptic-fraud-detection')
DATASET_DST = Path('data') / 'Elliptic++ Dataset'
DATASET_DST.parent.mkdir(parents=True, exist_ok=True)
if DATASET_SRC.exists():
    if DATASET_DST.exists():
        shutil.rmtree(DATASET_DST)
    shutil.copytree(DATASET_SRC, DATASET_DST)
else:
    print('WARNING: Add the elliptic-fraud-detection dataset.')

print(f"Working directory: {Path.cwd()}")
print(f"Dataset directory: {DATASET_DST}")


## 3. Set Seed

In [ ]:
import random
import numpy as np
import torch

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
print('Seeds set to 42')


## 4. Regenerate Splits (Safety)

In [ ]:
!python -m src.data.elliptic_loader --check

## 5. Run Temporal Shift Script

In [ ]:
!python scripts/run_m9_temporal_shift.py

## 6. Inspect Results

In [ ]:
import pandas as pd
from pathlib import Path
results_csv = Path('reports/m9_temporal_results.csv')
df = pd.read_csv(results_csv)
df


## 7. Package Artifacts

In [ ]:
import zipfile
from pathlib import Path
OUTPUT_ZIP = Path('m9_temporal_results.zip')
with zipfile.ZipFile(OUTPUT_ZIP, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
    if Path('reports/m9_temporal_results.csv').exists():
        zf.write('reports/m9_temporal_results.csv')
print(f'Bundled results into {OUTPUT_ZIP}')
